# MNIST Large Untrained Net Exc Inh

Derived from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb

In [104]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras import backend as K
#print("keras version:",tf.keras.__version__)

TensorFlow version: 2.8.2


## Load data

In [105]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Define model

In [106]:
def activationExcitatory(x):
    return K.maximum(x, 0)  #ReLU

def activationInhibitory(x):
    return -(K.maximum(x, 0))   #ReLU with negative output

def excitatoryNeuronInitializer(shape, dtype=None):
    return tf.math.abs(tf.random.normal(shape, dtype=dtype))

def inhibitoryNeuronInitializer(shape, dtype=None):
    #return tf.math.negative(tf.math.abs(tf.random.normal(shape, dtype=dtype)))
    return tf.math.abs(tf.random.normal(shape, dtype=dtype))


In [107]:
input_shape = (28, 28)
num_classes = 10

positiveWeightImplementation = True	#mandatory (only implementation)
if(positiveWeightImplementation):
    weightConstraint = tf.keras.constraints.non_neg()
    positiveWeightImplementationBiases = True   #ensure positive biases also
    if(positiveWeightImplementationBiases):
        biasConstraint = tf.keras.constraints.non_neg()
        positiveWeightImplementationBiasesLastLayer = False
        if(positiveWeightImplementationBiasesLastLayer):
            biasConstraintLastLayer = tf.keras.constraints.non_neg()
        else:
            biasConstraintLastLayer = None
    else:
        biasConstraint = None

generateUntrainedNetwork = False
if(generateUntrainedNetwork):
    #only train the last layer
    generateLargeNetwork = True
    numberOfHiddenLayers = 2
else:
    generateLargeNetwork = False
    numberOfHiddenLayers = 2

if(generateLargeNetwork):
    generateLargeNetworkRatio = 50
    layerRatio = generateLargeNetworkRatio
else:
    layerRatio = 1

x = tf.keras.layers.Input(shape=input_shape)
h0 = tf.keras.layers.Flatten()(x)

hLast = h0
if(numberOfHiddenLayers >= 1):
    h1E = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
    h1I = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
    h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
    h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
    h1 = tf.keras.layers.Concatenate()([h1E, h1I])
    hLast = h1
if(numberOfHiddenLayers >= 2):
    h2E = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h1)
    h2I = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h1)
    h2E = tf.keras.layers.Activation(activationExcitatory)(h2E)
    h2I = tf.keras.layers.Activation(activationInhibitory)(h2I)
    h2 = tf.keras.layers.Concatenate()([h2E, h2I])
    hLast = h2
if(numberOfHiddenLayers >= 3):
    h3E = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h2)
    h3I = tf.keras.layers.Dense(128*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h2)
    h3E = tf.keras.layers.Activation(activationExcitatory)(h3E)
    h3I = tf.keras.layers.Activation(activationInhibitory)(h3I)
    h3 = tf.keras.layers.Concatenate()([h3E, h3I])
    hLast = h3

if(generateUntrainedNetwork):
    hLast = tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x))(hLast)
y = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_constraint=weightConstraint, bias_constraint=biasConstraintLastLayer)(hLast)
model = tf.keras.Model(x, y)

#print(model.summary())
#model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
#evaluation accuracy: ? (with 1 or 2 hidden layers)


In [108]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [109]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

print(model.summary())

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten_11 (Flatten)           (None, 784)          0           ['input_12[0][0]']               
                                                                                                  
 dense_50 (Dense)               (None, 128)          100480      ['flatten_11[0][0]']             
                                                                                                  
 dense_51 (Dense)               (None, 128)          100480      ['flatten_11[0][0]']             
                                                                                           

In [110]:
for i, layer in enumerate(model.layers):
    weights = layer.get_weights()
    #print("weights = ", weights)

## Train model

In [111]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 12s 6ms/step - loss: 8.0635 - accuracy: 0.7226
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 1.4611 - accuracy: 0.8376
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6938 - accuracy: 0.8617
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3996 - accuracy: 0.8893
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3254 - accuracy: 0.9054


In [112]:
for i, layer in enumerate(model.layers):
    weights = layer.get_weights()
    #print("weights = ", weights)

## Evaluate model

In [113]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.3614 - accuracy: 0.8848 - 688ms/epoch - 2ms/step


[0.3614092171192169, 0.8848000168800354]

In [114]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [115]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0853469 , 0.0853469 , 0.08534728, 0.08540992, 0.0853469 ,
        0.08534729, 0.0853469 , 0.2318087 , 0.0853469 , 0.0853524 ],
       [0.08846652, 0.08824223, 0.1812067 , 0.09316508, 0.08811232,
        0.1010511 , 0.08941285, 0.08823032, 0.09398124, 0.08813164],
       [0.08537503, 0.23136264, 0.08537775, 0.08538289, 0.0853762 ,
        0.08538847, 0.08537746, 0.08553106, 0.08543802, 0.08539046],
       [0.16860355, 0.08853342, 0.09713642, 0.08910637, 0.08852744,
        0.08994108, 0.08852763, 0.11245561, 0.08857624, 0.08859228],
       [0.08868405, 0.08871052, 0.08880783, 0.08877318, 0.1480955 ,
        0.08883302, 0.08897909, 0.09024849, 0.08914426, 0.13972399]],
      dtype=float32)>